<a href="https://colab.research.google.com/github/vladgap/LLC/blob/main/Rotem%20LLC%20simulation%2010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import matplotlib.pyplot as plt

from scipy import optimize

In [2]:
!pip install MLNN
from MLNN import*

In [3]:
!wget -O LLC.py "https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_4_4.py"
import LLC

--2023-12-14 13:46:16--  https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_4_4.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21856 (21K) [text/plain]
Saving to: ‘LLC.py’

LLC.py              100%[===================>]  21.34K  --.-KB/s    in 0.002s  

2023-12-14 13:46:16 (13.1 MB/s) - ‘LLC.py’ saved [21856/21856]



In [4]:
def CopyPasteToArray(a):
  if a.startswith('\n'):
    b=a[1:]
  else:
    b=a
  if b.endswith('\n'):
    b=b[:-1]
  c=b.replace('\t',',')
  d=c.split('\n')
  f=[]
  for e in d:
    if e.replace(',','').replace('.','').isdigit(): # only digits no letters
      f.append(list(eval(e)))
    else:
      f.append(e.split(','))
  return f

# EQUIL data

## EQUIL_rotem_lit1
lit data with NN model (7 hidden sigmoids)

### Equilibrium literature data

In [5]:
data1=pd.DataFrame(data=[
[63.5,	1.3,	42.4,	15.0],
[58.7,	1.2,	36.9,	13.8],
[52.0, 1.0,	27.7,	11.8],
[45.5,	1.0,	18.9,	9.6],
[39.1,	1.4,	10.2,	8.3],
[31.5,	1.5,	5.1,	8.8],
[22.9,	2.0,	2.3,	9.0],
[12.6,	2.2,	0.90,	9.3],
[6.3,	2.4,	0.45,	9.45],
[0.01,	2.6,	0.01,	9.6],
],
columns=['H3PO4_aq', 'solvent_aq', 'H3PO4_org', 'water_org'])

In [6]:
data1.head()

,H3PO4_aq,solvent_aq,H3PO4_org,water_org
0,63.5,1.3,42.4,15.0
1,58.7,1.2,36.9,13.8
2,52.0,1.0,27.7,11.8
3,45.5,1.0,18.9,9.6
4,39.1,1.4,10.2,8.3


In [7]:
fig1=go.Figure()
[fig1.add_trace(go.Scatter(x=data1.H3PO4_aq, y=data1[data1.columns[i]], name=data1.columns[i])) for i in range(1,4)]
fig1.update_layout(title='', width=1200, height=600,)
fig1.show()

In [8]:
X1=np.array(data1)[:,[0]]
T1=np.array(data1)[:,[1,2,3]]

### Model
"Model for Rotem LLC simulation 1"

In [9]:
scaler_X1 = preprocessing.StandardScaler()
scaler_T1 = preprocessing.StandardScaler()
scaler_X1.mean_, scaler_X1.scale_ = [33.211], [21.13387066]
scaler_T1.mean_, scaler_T1.scale_ = [1.66,  14.486, 10.465], [0.56071383, 15.23739033,  2.16703138]

bp1=VectorBackProp(layers=[1,7,3], hidden_activation = 'sigmoid')
manual_best_weights_for_now2=[[[-4.215621245080982,
   -8.235086763118087,
   -1.6251342150606451,
   -6.9206357615871035,
   -3.4092427169369826,
   -7.4263402914028775,
   -3.7003458539301977]],
 [[0.8764395040443358,
   3.9145850159876945,
   -1.5013585830931442,
   -1.194371163565281,
   4.118411550251139,
   5.892242255497829,
   -6.755593628941706]],
 [[-2.6609909047049918, -0.44480776958732143, 1.9256391063013463],
  [2.566022222516618, -0.5042582093932455, -1.432212975592254],
  [1.405004033981333, -0.2988952466228391, 0.5807591718257896],
  [2.0692859230668117, -0.0780871568942863, -0.5145373287488922],
  [-0.7989625307881565, -1.957459884661486, -2.8991426864622065],
  [-0.5821795598385298, -0.22284877970127598, -0.9077232625087237],
  [1.619488070885094, 0.03697105802710269, 0.008613201857731975]],
 [[-0.4010006364699687, 2.4687577407660806, 2.995012357373033]]]
bp1.import_weights(manual_best_weights_for_now2)

In [10]:
class EQUIL_rotem_lit1: # literature data for pentanol 25C
# gets xout [%solvent_aq, %H3PO4_aq]
# calc xout , yout [%water_org, %H3PO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4], [%solv, %H3PO4], [%solv, %H3PO4], ...]
        # the model predicts %solvent_aq, %H3PO4_aq, %water_org from %H3PO4_aq
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        pred=scaler_T1.inverse_transform(bp1.run(scaler_X1.transform(x[:,[1]])))
        # prediction is column vector of lists [%solvent_aq, %H3PO4_aq, %water_org]

        self.xout = np.array([pred[:,0], x[:,1]]).T # [[%solvent_aq, %H3PO4_aq], ...]
        self.yout = np.array([pred[:,2], pred[:,1]]).T # [[%water_org, %H3PO4_org], ...]

## EQUIL_rotem_washing1
Rotem solvent, estimation by polynoms

In [11]:
class EQUIL_rotem_washing1: # prepared using purified and washed solvents
# gets xout [%solvent_aq, %H3PO4_aq]
# calc xout , yout [%water_org, %H3PO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4], [%solv, %H3PO4], [%solv, %H3PO4], ...]
        y=np.zeros_like(x)
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # y = A + B*x + C*x^2 + D*x^3

        # %solvent_aq
        D,C,B,A = -3.78E-06,8.15E-04,- 5.73E-02,2.45E+00
        x[:,0] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3

        # %water_org
        D,C,B,A = 4.45E-05,- 1.56E-03,- 1.84E-02,7.58E+00
        y[:,0] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3

        # %H3PO4_org
        # D,C,B,A = 2.00E-04,- 4.86E-03,7.07E-02,2.73E-02
        # y[:,1] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3
        G,F,E,D,C,B,A = -1.97E-09,2.02E-07,- 2.95E-06,- 3.83E-05,2.60E-03,2.38E-02,1.70E-02
        y[:,1] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3 + E*x[:,1]**4 + F*x[:,1]**5 + G*x[:,1]**6

        self.xout = np.array([x[:,0], x[:,1]]).T # [[%solvent_aq, %H3PO4_aq], ...]
        self.yout = np.array([y[:,0], y[:,1]]).T # [[%water_org, %H3PO4_org], ...]

In [12]:
# EQUIL_rotem_washing1([[2,20],[1.5,30],[1.3,40]]).xout

## EQUIL_rotem_washing2
Rotem solvent, full profile, sulfate

In [13]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org
4.15	0.147	0.0733	9.6	2.5	0.0626
14.9	0.0806	0.359	9.5	2.2	0.0852
28.6	0.0329	1.68	9.3	1.9	0.0787
45.7	0.0106	8.31	8.5	1.6	0.0664
48.2	0.0078	10.7	8.5	1.3	0.0631
49.4	0.0058	12.2	8.7	1.0	0.0641
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data2=pd.DataFrame(data[1:], columns=data[0])
data2

,H3PO4_aq,SO4_aq,H3PO4_org,water_org,solvent_aq,SO4_org
0,4.15,0.1470,0.0733,9.6,2.5,0.0626
1,14.90,0.0806,0.3590,9.5,2.2,0.0852
2,28.60,0.0329,1.6800,9.3,1.9,0.0787
3,45.70,0.0106,8.3100,8.5,1.6,0.0664
4,48.20,0.0078,10.7000,8.5,1.3,0.0631
5,49.40,0.0058,12.2000,8.7,1.0,0.0641


In [14]:
X2=np.array(data2)[:,[0,1]]
T2_1=np.array(data2)[:,[2]]
T2_2=np.array(data2)[:,[3]]
T2_3=np.array(data2)[:,[4]]
T2_4=np.array(data2)[:,[5]]

scaler_X2 = preprocessing.StandardScaler().fit(X2)
scaler_T2_1 = preprocessing.StandardScaler().fit(T2_1)
scaler_T2_2 = preprocessing.StandardScaler().fit(T2_2)
scaler_T2_3 = preprocessing.StandardScaler().fit(T2_3)
scaler_T2_4 = preprocessing.StandardScaler().fit(T2_4)

X_sc2 = scaler_X2.transform(X2)
# T_sc2_1 = scaler_T2_1.transform(T2_1)
# T_sc2_2 = scaler_T2_1.transform(T2_2)
# T_sc2_3 = scaler_T2_1.transform(T2_3)
# T_sc2_4 = scaler_T2_1.transform(T2_4)

In [15]:
class EQUIL_rotem_washing2: # from full profile, sulfate
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp2_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_1=[[[0.1735491429461148, 1.8140938189194906],
  [-0.6530409129171293, -0.6441814583921941]],
 [[0.7411897128053353, -1.2033881701371891]],
 [[0.3516552639162472], [1.6547118904069311]],
 [[-1.0860308848681413]]]
        bp2_1.import_weights(manual_weights2_1)
        predics2_1=abs(scaler_T2_1.inverse_transform(bp2_1.run(scaler_X2.transform(x[:,[1,2]]))))

        # water_org
        bp2_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_2=[[[-1.356083366626272, 2.1006456906712234],
  [-0.6309805208401018, -0.5374527149916659]],
 [[1.0515374304634164, -2.3182953246087115]],
 [[1.4841383017903245], [3.153523627853289]],
 [[-1.7074908699001965]]]
        bp2_2.import_weights(manual_weights2_2)
        predics2_2=abs(scaler_T2_2.inverse_transform(bp2_2.run(scaler_X2.transform(x[:,[1,2]]))))

        # solvent_aq
        bp2_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_3=[[[2.0994823528593702, -1.2293957752203057],
  [-0.6972915446623457, 0.5216771334283966]],
 [[-2.3459563115354527, 1.775004898861346]],
 [[-3.1726827033207408], [0.4025651395234753]],
 [[-0.4638947231253712]]]
        bp2_3.import_weights(manual_weights2_3)
        predics2_3=abs(scaler_T2_3.inverse_transform(bp2_3.run(scaler_X2.transform(x[:,[1,2]]))))

        # SO4_org
        bp2_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_4=[[[-2.7308241499437758, 1.8373933992350313],
  [-2.021898044039464, -0.8845457922101067]],
 [[1.0694417308325381, 0.08920200363835196]],
 [[2.7835653102552724], [1.6175935763910405]],
 [[-4.969235753353272]]]
        bp2_4.import_weights(manual_weights2_4)
        predics2_4=abs(scaler_T2_4.inverse_transform(bp2_4.run(scaler_X2.transform(x[:,[1,2]]))))

        self.xout = np.array([predics2_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics2_2[:,0], predics2_1[:,0], predics2_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

## EQUIL_rotem_washing3
ניסויי הפרדה עם ממס מטוהר

In [16]:
Xscale=[21.97576471,  0.05575882], [15.2853469,   0.05047379]
Tscale=[1.89941176, 11.19176471,  1.4,         0.02921412], [2.40216868, 1.27085536, 0.17875386, 0.01997862]

scaler_X3 = preprocessing.StandardScaler().fit([[0,0]])
scaler_X3_2 = preprocessing.StandardScaler().fit([[0,0]]) # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_2 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_3 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_4 = preprocessing.StandardScaler().fit([[0]])

scaler_X3.mean_, scaler_X3.scale_ = Xscale[0], Xscale[1]
scaler_X3_2.mean_, scaler_X3_2.scale_ = [27.03608333,  0.0474    ], [14.68499343,  0.04213688] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1.mean_, scaler_T3_1.scale_=[Tscale[0][0]], [Tscale[1][0]]
# scaler_T3_2.mean_, scaler_T3_2.scale_=[Tscale[0][1]], [Tscale[1][1]]
scaler_T3_2.mean_, scaler_T3_2.scale_=[10.81], [1.3091664] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_3.mean_, scaler_T3_3.scale_=[Tscale[0][2]], [Tscale[1][2]]
scaler_T3_4.mean_, scaler_T3_4.scale_=[Tscale[0][3]], [Tscale[1][3]]

In [17]:
class EQUIL_rotem_washing3: # ניסויי הפרדה עם ממס מטוהר
# מודל מקורי
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]])))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        predics3_2=scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]]))) # מים בקלה חושבו רק על נתונים "טובים

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]])))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]])))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [18]:
class EQUIL_rotem_washing3_2: # ניסויי הפרדה עם ממס מטוהר
# מודל עם מים בקלה 12% גורף
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        # predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]])))) # מים בקלה חושבו רק על נתונים "טובים
        predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [19]:
class EQUIL_rotem_washing3_3: # ניסויי הפרדה עם ממס מטוהר
# מודל עם מים בקלה 12% גורף
# שינוי במשקלים של ממס בכבדה. שיהיה יותר אופקי
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        # predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]])))) # מים בקלה חושבו רק על נתונים "טובים
        predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu') # modified solvent_aq weights
        manual_weights3_3=[[[-7.637254337641781, -0.490322880885377],
  [-1.8058063735873857, -0.10159866035465558]],
 [[-0.843126492292387, -2.3299686091075773]],
 [[0.2643116291427324], [0.4479958576488306]],
 [[-0.7056420231884761]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

# Cases

## Literature data

### 1 - Washing



In [20]:
bat1_2=LLC.Battery(stages_num=5, Oin=90, Ain=29, yin=[5,12], xin=[5,0], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=[1, 0.7], entrainment_perc_in=[2,5], entrainment_comp_in=[[5,5],[10,10]], entrainment_perc_out=[3,4])
# bat1_2=LLC.Battery(4, 87.6, 112.4, [0, 0], [0, 32.56], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=.7, entrainment_perc_out=10)

In [21]:
bat1_2.runs

35

In [22]:
table1_2=LLC.BatteryTableFull(bat1_2).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",90.000000,89.055359,85.475686,83.948464,83.262505
"y1 in, %",5.000000,8.829622,9.079322,9.354564,9.494674
"y2 in, %",12.000000,6.974399,3.352100,1.529652,0.668417
"Aq in, ton/hr",34.382667,30.833850,29.321083,28.640994,29.000000
"x1 in, %",2.075036,2.268582,2.424578,2.529920,5.000000
"x2 in, %",19.948914,10.447329,4.783224,1.865826,0.000000
"Org out, ton/hr",89.055213,85.474791,83.948421,83.262549,83.906581
"y1 out, %",8.829647,9.079543,9.354583,9.494752,9.558549
"y2 out, %",6.974221,3.350772,1.529553,0.667935,0.281837
"Aq out, ton/hr",34.452985,34.379990,30.833663,29.320314,28.640963


In [23]:
bat=bat1_2
fig1_3 = go.Figure()
n=1 #H3PO4
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig1_3.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig1_3.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig1_3.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig1_3.update_layout(title='', width=1200, height=600,)
fig1_3.show()

In [24]:
fig1_4=go.Figure()
current_data = data1
current_table = table1_2
[fig1_4.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig1_4.update_xaxes(title_text="x, %", range=[20, 40])
# fig1_4.update_yaxes(title_text="y, %", range=[0, 15])
fig1_4.update_layout(title='', width=1200, height=600,)
fig1_4.show()

### 2 - Extraction

In [25]:
bat2=LLC.Battery(stages_num=7, Oin=90, Ain=29, yin=[5,0], xin=[0,40], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=[[.8, 1], [0.5, .8], [.6,.9]])

In [26]:
bat2.runs

32

In [27]:
bat2.eff

array([[1. , 1. ],
       [1. , 0.8],
       [1. , 0.9],
       [1. , 1. ],
       [1. , 1. ],
       [1. , 1. ],
       [1. , 1. ]])

In [28]:
table2=LLC.BatteryTableFull(bat2).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6,Stage 7
"Org in, ton/hr",90.000000,96.478357,97.003163,97.604162,98.301236,99.074305,100.087489
"y1 in, %",5.000000,8.990625,8.935682,8.912865,8.882158,8.817497,8.685067
"y2 in, %",0.000000,2.349489,2.904449,3.490731,4.166842,4.938127,5.944875
"Aq in, ton/hr",24.318538,24.845713,25.444959,26.115418,26.910527,27.926923,29.000000
"x1 in, %",1.844670,1.739844,1.626737,1.517951,1.440225,1.414601,0.000000
"x2 in, %",26.108895,27.756173,29.406546,31.196803,33.213964,35.770416,40.000000
"Org out, ton/hr",96.477896,97.000448,97.603301,98.294507,99.079764,100.083483,101.184763
"y1 out, %",8.990690,8.935841,8.912904,8.882584,8.816970,8.685311,8.455164
"y2 out, %",2.348938,2.901511,3.489799,4.159975,4.943057,5.943249,7.483957
"Aq out, ton/hr",17.840642,24.323621,24.844820,25.425072,26.132000,26.917746,27.902726


In [29]:
bat=bat2
fig2_1 = go.Figure()
n=1 #H3PO4
fig2_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig2_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig2_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig2_1.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig2_1.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig2_1.update_layout(title='', width=1200, height=600,)
fig2_1.show()

In [30]:
fig2_2=go.Figure()
current_data = data1
current_table = table2
[fig2_2.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig2_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig2_2.update_yaxes(title_text="y, %", range=[0, 15])
fig2_2.update_layout(title='', width=1200, height=600,)
fig2_2.show()

## Washing data
Rotem solvent, estimation by polynoms

### 3 - Washing1

In [31]:
bat3=LLC.Battery(stages_num=7, Oin=204, Ain=33.3, yin=[7.7,14.4], xin=[0,0], EQUIL=EQUIL_rotem_washing1)#, convergence=5e-4, eff=[1, 0.7], entrainment_perc_in=[2,5], entrainment_comp_in=[[5,5],[10,10]], entrainment_perc_out=[3,4])

100 iterations!


In [32]:
bat3.runs

100

In [33]:
table3=LLC.BatteryTable(bat3).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6,Stage 7
"Org in, ton/hr",204.000000,195.337881,183.274167,174.324357,172.269056,172.061557,172.039704
"y1 in, %",7.700000,7.577722,7.051630,6.835923,7.296174,7.533943,7.574142
"y2 in, %",14.400000,11.103383,6.305719,2.041970,0.445752,0.079372,0.024633
"Aq in, ton/hr",57.511093,45.438347,36.496542,34.438299,34.231131,34.209249,33.300000
"x1 in, %",1.246025,1.459461,1.968391,2.331133,2.432301,2.448016,0.000000
"x2 in, %",37.664462,25.356147,9.677278,2.138886,0.310254,0.034642,0.000000
"Org out, ton/hr",195.340842,183.269377,174.326820,172.268784,172.061581,172.039712,171.130451
"y1 out, %",7.577855,7.051432,6.835834,7.296424,7.533887,7.574151,7.579360
"y2 out, %",11.104483,6.303637,2.043380,0.445333,0.079451,0.024622,0.017829
"Aq out, ton/hr",66.170251,57.506851,45.443890,36.493872,34.438606,34.231094,34.209253


In [34]:
bat=bat3
fig3 = go.Figure()
n=1 #H3PO4
fig3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig3.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig3.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig3.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig3.update_layout(title='', width=1200, height=600,)
fig3.show()

In [35]:
fig3_2=go.Figure()
# current_data = data1
current_table = table3
# [fig1_4.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig3_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig3_2.update_yaxes(title_text="y, %", range=[0, 15])
fig3_2.update_layout(title='', width=1200, height=600,)
fig3_2.show()

### 4 - Washing2

In [36]:
bat4=LLC.Battery(stages_num=7, Oin=204, Ain=33.3, yin=[7.7,14.4], xin=[0,0], EQUIL=EQUIL_rotem_washing1, convergence=5e-4, eff=[1, 0.9], entrainment_perc_in=[2,0], entrainment_comp_in=[[5,5],[0,0]], entrainment_perc_out=[3,4])

In [37]:
bat4.runs

13

In [38]:
table4=LLC.BatteryTableFull(bat4).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6,Stage 7
"Org in, ton/hr",204.000000,197.476924,192.054619,184.980696,177.693848,173.096470,171.797120
"y1 in, %",7.700000,7.639709,7.413024,7.107238,6.848766,6.914708,7.272929
"y2 in, %",14.400000,11.892679,9.921331,7.179893,4.041803,1.668367,0.608861
"Aq in, ton/hr",59.968484,54.876756,47.942418,40.551758,36.028255,34.633532,33.300000
"x1 in, %",1.195117,1.234671,1.330063,1.567682,1.941366,2.272553,0.000000
"x2 in, %",42.683779,38.641698,31.823762,21.122384,10.318621,3.244250,0.000000
"Org out, ton/hr",197.489684,192.098515,185.012627,177.569184,173.104408,171.776807,169.156745
"y1 out, %",7.640308,7.414765,7.108539,6.846031,6.913954,7.285448,7.505317
"y2 out, %",11.897083,9.937810,7.192845,3.994027,1.672885,0.584278,0.169015
"Aq out, ton/hr",64.454662,60.203450,54.916699,47.892005,40.573566,35.940507,34.634218


In [39]:
bat=bat4
fig4 = go.Figure()
n=1 #H3PO4
fig4.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig4.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig4.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig4.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig4.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig4.update_layout(title='Operation Data', width=1200, height=600,)
fig4.show()

In [40]:
fig4_2=go.Figure()
# current_data = data1
current_table = table4
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig4_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig4_2.update_yaxes(title_text="y, %", range=[0, 15])
fig4_2.update_layout(title='Equilibrium Data', width=1200, height=600,)
fig4_2.show()

## Rotem washing with sulfate

### 5 - Washing1

In [41]:
EQUIL_rotem_washing2([[2.2,14.9,.0806]]).yout

array([[9.56542351, 0.35871824, 0.08520132]])

In [42]:
test1 = LLC.Stage(90, 29, [10, 14, .07], [2, 0.1, .02], EQUIL_rotem_washing2, eff=1)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)

In [43]:
test1.runs

10

In [44]:
test1.output()

Oout= 74.5042053393592
yout= [8.60822308 0.09877921 0.02661528]
yout_tag= [8.60822308 0.09877921 0.02661528]  %water_org_equilib, %solubles_equilib
Aout= 44.49579466064079
xout= [ 2.06758444 28.21705147  0.11005343]
equilibrium efficiency= [1. 1. 1.]
error= [4.11296949e-05 5.50745531e-05 6.80630206e-05]
convergence= [-0.00040428 -0.00022316 -0.00016548]


In [45]:
test1 = LLC.Stage(90, 29, [8.60822308, 0.09877921, 0.02661528], [2.06758444, 28.21705147, 0.11005343], EQUIL_rotem_washing2, eff=1)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)

In [46]:
bat5_1=LLC.Battery(stages_num=4, Oin=90, Ain=29, yin=[8.60822308, 1, 0.05], xin=[2.06758444, 30, 0.3], EQUIL=EQUIL_rotem_washing2)

In [47]:
bat5_1.runs

6

In [48]:
table5=LLC.BatteryTable(bat5_1).data

,Stage 1,Stage 2,Stage 3,Stage 4
"Org in, ton/hr",90.000000,88.607990,88.615281,88.615270
"y1 in, %",8.608223,8.215397,8.216589,8.216587
"y2 in, %",1.000000,0.070425,0.076451,0.076442
"y3 in, %",0.050000,0.026314,0.026386,0.026385
"Aq in, ton/hr",29.114540,29.121741,29.121729,29.000000
"x1 in, %",2.442020,2.442172,2.442171,2.067584
"x2 in, %",29.864179,29.874934,29.874918,30.000000
"x3 in, %",0.298710,0.298857,0.298856,0.300000
"Org out, ton/hr",88.607990,88.615281,88.615270,88.493540
"y1 out, %",8.215397,8.216589,8.216587,8.216587


In [49]:
bat=bat5_1
fig5 = go.Figure()
n=1 #H3PO4
fig5.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig5.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig5.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig5.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig5.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig5.update_layout(title='Operation Data', width=1200, height=600,)
fig5.show()

In [50]:
fig_equil2 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.SO4_org, mode='markers', name='SO4_org'), 2, 2)

bat=bat5_1
# H3PO4_org
fig_equil2.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', name='Efficiency'), 1, 1)
fig_equil2.add_trace(go.Scatter3d(x=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])), y=np.concatenate((bat.xout_list[:,2], [bat.xin_list[-1,2]])), z=np.concatenate((bat.yin_list[:,1], [bat.yout_list[-1,1]])),
                                mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),
                                                                                                                            np.concatenate((bat.xout_list[:,2], [bat.xin_list[-1,2]])),
                                                                                                                            np.concatenate((bat.yin_list[:,1], [bat.yout_list[-1,1]])))],
                                textposition ='bottom center', name='Operation', marker_color='green'),1,1)

fig_equil2.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil2.update_layout(
    scene1_zaxis_range=[0,15],
    scene2_zaxis_range=[8,10],
    scene3_zaxis_range=[0,3],
    scene4_zaxis_range=[.05,.1],
    )

fig_equil2.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))

fig_equil2.show()

## Rotem washing LLC tests

### data

In [51]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org	water_org-original
0.011	0.0932	0.072	8.69	1.70	0.0186	12.09
10.325	0.0017	0.229	10.66	1.52	0.02	10.66
10.380	0.0990	0.269	8.31	1.50	0.0208	10.41
19.821	0.0031	0.635	11.3	1.34	0.01884	11.3
19.710	0.0928	0.773	10.36	1.32	0.0195	12.34
28.710	0.0057	1.435	11.96	1.28	0.01978	11.96
27.882	0.0858	1.325	9.7	1.34	0.0222	11.1
35.611	0.0071	2.871	11.94	1.28	0.036	11.94
35.887	0.0846	2.981	12.02	1.24	0.0374	12.6
43.065	0.0082	5.328	12.12	1.28	0.057	12.12
43.065	0.0802	5.604	10.38	1.06	0.0608	11.7
49.966	0.0074	8.723	12.28	1.28	0.0872	12.28
0.017	0.1540	0.042	11.48	1.46	0.0164	13.28
4.831	0.0020	0.146	12.7	1.78	0.01302	12.6
4.941	0.1078	0.173	12.06	1.66	0.0139	11.84
19.600	0.0033	0.828	11.84	1.38	0.01688	11.42
19.766	0.1120	0.856	12.46	1.38	0.01832	11.2
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data3=pd.DataFrame(data[1:], columns=data[0])
data3_names=CopyPasteToArray('''
wp12	wp21	wp22	wp31	wp32	wp41	wp42	wp51	wp52	wp61	wp62	wp71	wp13	ww11	ww12	ww21	ww22
''')

In [52]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh3 = x_mesh.flatten().reshape(31*31,1)
y_mesh3 = y_mesh.flatten().reshape(31*31,1)
mesh3=np.concatenate((x_mesh3, y_mesh3), axis=1)
# x_mesh[:5]

In [53]:
a=np.concatenate((np.zeros([len(mesh3),1]),mesh3), axis=1)

In [54]:
mesh_predict_x=np.array([EQUIL_rotem_washing3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3([row]).yout for row in a])
mesh_predics3_1=mesh_predict_y[:,:,1]
mesh_predics3_2=mesh_predict_y[:,:,0]
mesh_predics3_3=mesh_predict_x[:,:,0]
mesh_predics3_4=mesh_predict_y[:,:,2]

### test

In [55]:
EQUIL_rotem_washing3([[2.2,10.38,.099]]).yout

array([[9.02976303, 0.5493118 , 0.01802152]])

In [56]:
test1 = LLC.Stage(90, 29, [10, 14, .07], [0, 0.1, .02], EQUIL_rotem_washing3, eff=.8)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)
test1.runs

3

In [57]:
test1.output()

Oout= 78.32179881063443
yout= [9.60912967 3.76230026 0.03045747]
yout_tag= [9.60912967 1.20287533 0.02057183]  %water_org_equilib, %solubles_equilib
Aout= 40.67820118936556
xout= [ 1.25110892 23.80920178  0.11050305]
equilibrium efficiency= [1.  0.8 0.8]
error= [ 7.39425190e-07 -9.69302553e-04 -2.30061189e-04]
convergence= [-2.05215689e-04 -7.82960672e-04 -8.59080721e-05]


### 6 - Washing1 - water_org model

In [58]:
bat6_1=LLC.Battery(stages_num=5, Oin=90, Ain=29, yin=[10, 14, .07], xin=[0, 0.1, .02], EQUIL=EQUIL_rotem_washing3, eff=.7)
bat6_1.runs

21

In [59]:
table6_1=LLC.BatteryTable(bat6_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",90.000000,80.315656,77.402606,76.362880,75.972655
"y1 in, %",10.000000,9.556306,9.657001,9.716974,9.747376
"y2 in, %",14.000000,5.219303,1.992958,0.760381,0.281593
"y3 in, %",0.070000,0.036196,0.023426,0.018414,0.016422
"Aq in, ton/hr",34.016975,31.103606,30.063793,29.673346,29.000000
"x1 in, %",1.626419,1.610304,1.602890,1.599657,0.000000
"x2 in, %",12.201220,4.825139,1.792163,0.579511,0.100000
"x3 in, %",0.067926,0.039113,0.026924,0.021934,0.020000
"Org out, ton/hr",80.315643,77.402535,76.362846,75.972510,75.299319
"y1 out, %",9.556313,9.657020,9.716978,9.747382,9.761492


In [60]:
fig_equil6_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat6_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil6_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil6_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil6_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil6_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil6_1.update_scenes(camera_projection_type="orthographic")

fig_equil6_1.show()

### 7 - Washing2 - water_org=12

In [61]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [62]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [63]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_2([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_2([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [64]:
bat7_1=LLC.Battery(stages_num=5, Oin=90, Ain=29, yin=[10, 14, .07], xin=[0, 0.1, .02], EQUIL=EQUIL_rotem_washing3_2, eff=.7)
bat7_1.runs

21

In [65]:
table7_1=LLC.BatteryTable(bat7_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",90.000000,82.794901,79.586005,78.421448,77.976314
"y1 in, %",10.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",14.000000,5.267622,2.018177,0.773599,0.288009
"y3 in, %",0.070000,0.036412,0.023539,0.018473,0.016451
"Aq in, ton/hr",34.530811,31.321228,30.156551,29.711021,29.000000
"x1 in, %",1.627135,1.610703,1.603071,1.599715,0.000000
"x2 in, %",12.499448,4.982271,1.859997,0.601322,0.100000
"x3 in, %",0.069119,0.039751,0.027203,0.022024,0.020000
"Org out, ton/hr",82.794879,79.585875,78.421405,77.976065,77.265306
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000


In [66]:
fig_equil7_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat7_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil7_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil7_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil7_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil7_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil7_1.update_scenes(camera_projection_type="orthographic")

fig_equil7_1.show()

### 8 - Washing3 - compare with plant data

Profile data

In [67]:
data=CopyPasteToArray('''
x	y
0	0.0531
3.01	0.26
10.8	1.22
20.69	6.02
33.12	7.77
34.9	8.86
35.81	10.3
''')
profile_data1=pd.DataFrame(data[1:], columns=data[0])

Plant data

In [68]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6

PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

Purified solvent flow = 159.2 ton/hr,   Washed solvent flow = 134.2 ton/hr   P2O5 recovery = 98.5%


In [69]:
summary_table8=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table8

,Plant data
PS_flow,159.196491
PS_P2O5,11.6
WS_flow,134.196491
WS_P2O5,0.2
W_flow,23.4
Prod_flow,48.4
Prod_P2O5,37.6
P2O5_recovery,98.5%
Stages_num,6
Efficiency,---


[0.76853034 3.57952069] root,
[0.75617284 3.26131687] direct,
[0.77263374 3.61111111] direct *10,
[0.77057613 3.48536808 2.34910837] direct entrain

In [70]:
bat8_1=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=.77, convergence=1E-3, entrainment_perc_out=[3.6,0])
bat8_1.runs

50

In [71]:
bat=bat8_1
summary_table8['Model data-0.75-3']=[bat.Oin,bat.yin[1]/98*71,bat.Oout,bat.yout[1]/98*71,bat.Ain,bat.Aout,bat.xout[1]/98*71,f"{bat.Aout*bat.xout[1]/(bat.Oin*bat.yin[1]):.1%}", f"{bat.stages_num:.0f}", bat.eff[0,1], str(bat.EQUIL)[17:-2]]
summary_table8

,Plant data,Model data-0.75-3
PS_flow,159.196491,159.196491
PS_P2O5,11.6,11.6
WS_flow,134.196491,129.746145
WS_P2O5,0.2,0.202226
W_flow,23.4,23.4
Prod_flow,48.4,48.176587
Prod_P2O5,37.6,37.563144
P2O5_recovery,98.5%,98.0%
Stages_num,6,6
Efficiency,---,0.77


In [72]:
table8_1=LLC.BatteryTable(bat8_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6
"Org in, ton/hr",159.196491,148.473930,141.232149,134.408934,131.976964,130.845320
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,10.891694,7.023403,3.017502,1.440431,0.685758
"y3 in, %",0.070000,0.088345,0.069770,0.037963,0.024245,0.019070
"Aq in, ton/hr",42.791837,35.326272,28.214773,25.715555,24.536636,23.400000
"x1 in, %",1.272657,1.272959,1.273546,1.502382,1.702555,0.000000
"x2 in, %",43.058998,32.899565,18.145626,8.801496,3.237896,0.000000
"x3 in, %",0.287036,0.267521,0.159355,0.080895,0.040186,0.020000
"Org out, ton/hr",148.466932,141.238996,134.400139,131.977765,130.847334,129.746145
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000


In [73]:
## P2O5
bat=bat8_1
summary_table=summary_table8
fig8_1 = go.Figure()
n=1 #H3PO4
fig8_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig8_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig8_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig8_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig8_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig8_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig8_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig8_1.update_layout(title='', width=1200, height=600,)
fig8_1.show()

In [74]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [75]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [76]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [77]:
fig_equil8_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat8_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil8_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil8_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil8_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil8_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil8_1.update_scenes(camera_projection_type="orthographic")

fig_equil8_1.show()

### 10 - Washing5 - optimize.direct()

Plant data

In [78]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6

PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

Purified solvent flow = 159.2 ton/hr,   Washed solvent flow = 134.2 ton/hr   P2O5 recovery = 98.5%


In [79]:
summary_table10=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table10

,Plant data
PS_flow,159.196491
PS_P2O5,11.6
WS_flow,134.196491
WS_P2O5,0.2
W_flow,23.4
Prod_flow,48.4
Prod_P2O5,37.6
P2O5_recovery,98.5%
Stages_num,6
Efficiency,---


#### slope(entr,eff)

נמצא שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
מרובע. מחשבים פרמטרים של פונקציה זאת

slope = a + b * entr + c * eff + d * eff^2

בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]

האופטימיזציה הופכת לאחת עם משתנה בודד.

In [80]:
## P2O5
summary_table=summary_table10
fig10_1 = go.Figure()
n=1 #H3PO4

for entr in [2.6, 3.6, 4.6]:
  for eff in [0.67,0.77,0.87]:
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    print ('entr:', entr, '  eff:', eff, '  slope:', (bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1]))
    fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          # text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='entr:'+str(entr)+', '+'eff:'+str(eff)))

bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=.77, convergence=1E-3, entrainment_perc_out=[3.6,0])
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

entr: 2.6   eff: 0.67   slope: 0.30925362450697086
entr: 2.6   eff: 0.77   slope: 0.311369197747131
entr: 2.6   eff: 0.87   slope: 0.3123725970713457
entr: 3.6   eff: 0.67   slope: 0.30059447877441686
entr: 3.6   eff: 0.77   slope: 0.30342971119296425
entr: 3.6   eff: 0.87   slope: 0.3044234949937369
entr: 4.6   eff: 0.67   slope: 0.292340334680572
entr: 4.6   eff: 0.77   slope: 0.295040031531457
entr: 4.6   eff: 0.87   slope: 0.29627613782255785


In [81]:
slopes=[]
for entr in [3.6, 2.6, 4.6]: # לינארי -- מספיק 2
  for eff in [0.77,0.67,0.87]: # פורבולה -- צריך 3
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    slope=(bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1]) # שיפוע של קו הפעולה
    slopes.append([entr, eff, slope])
    # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
arr=np.array(slopes)
arr

array([[3.6       , 0.77      , 0.30342971],
       [3.6       , 0.67      , 0.30059448],
       [3.6       , 0.87      , 0.30442349],
       [2.6       , 0.77      , 0.3113692 ],
       [2.6       , 0.67      , 0.30925362],
       [2.6       , 0.87      , 0.3123726 ],
       [4.6       , 0.77      , 0.29504003],
       [4.6       , 0.67      , 0.29234033],
       [4.6       , 0.87      , 0.29627614]])

In [82]:
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=arr[:,2], mode = "markers"))
fig.show()

In [83]:
arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]]
arr_mod

array([[1.        , 3.6       , 0.77      , 0.5929    , 0.30342971],
       [1.        , 3.6       , 0.67      , 0.4489    , 0.30059448],
       [1.        , 3.6       , 0.87      , 0.7569    , 0.30442349],
       [1.        , 2.6       , 0.77      , 0.5929    , 0.3113692 ],
       [1.        , 2.6       , 0.67      , 0.4489    , 0.30925362],
       [1.        , 2.6       , 0.87      , 0.7569    , 0.3123726 ],
       [1.        , 4.6       , 0.77      , 0.5929    , 0.29504003],
       [1.        , 4.6       , 0.67      , 0.4489    , 0.29234033],
       [1.        , 4.6       , 0.87      , 0.7569    , 0.29627614]])

In [84]:
rows=[0,1,2,3]
A=arr_mod[rows,0:4]
B=arr_mod[rows,4]
C=np.dot(np.linalg.inv(A),B) # מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות
C

array([ 0.26268041, -0.00793949,  0.16093662, -0.09207243])

In [85]:
pred=np.dot(arr_mod[:,0:4],C)
pred

array([0.30342971, 0.30059448, 0.30442349, 0.3113692 , 0.30853397,
       0.31236298, 0.29549022, 0.29265499, 0.29648401])

In [86]:
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=arr[:,2], mode = "markers"))
# fig.add_trace(go.Scatter3d(x=arr[rows,0], y=arr[rows,1], z=pred[rows], mode = "markers"))
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=pred, mode = "markers"))

fig.show()

#### together

Plant data

In [87]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6
PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

slope_proc=(PS_P2O5-WS_P2O5)/Prod_P2O5

Purified solvent flow = 159.2 ton/hr,   Washed solvent flow = 134.2 ton/hr   P2O5 recovery = 98.5%


In [88]:
summary_table10_1=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table10_1

,Plant data
PS_flow,159.196491
PS_P2O5,11.6
WS_flow,134.196491
WS_P2O5,0.2
W_flow,23.4
Prod_flow,48.4
Prod_P2O5,37.6
P2O5_recovery,98.5%
Stages_num,6
Efficiency,---


Slope(entrain,eff)

נמצא שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
מרובע. מחשבים פרמטרים של פונקציה זאת

slope = a + b * entr + c * eff + d * eff^2

בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]

האופטימיזציה הופכת לאחת עם משתנה בודד.

In [89]:
slopes=[]
entr_initial, eff_init = 3.6, 0.77
for entr in [entr_initial, entr_initial*.75]: # לינארי -- מספיק 2
  for eff in [eff_init,eff_init*.85,eff_init*1.1]: # פורבולה -- צריך 3
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    slope=(bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1])
    slopes.append([entr, eff, slope])
    # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
arr=np.array(slopes)
arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]] # entrainment approximated to porabola

In [90]:
rows=[0,1,2,3] # 4 rows for 4 unknowns
A=arr_mod[rows,0:4]
B=arr_mod[rows,4]
C=np.dot(np.linalg.inv(A),B) # פתרון מערכת משוואות.  מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות

optimize

optimization with 1 parameter

במקום להריץ אופטימיזציה עם 2 משתנים: יעילות וסחיפות, זה רץ על פרמטר בודד -- יעילות. הסחיפות מחושבות מתוך תלות בין סחיפות ושיפוע קו פעולה. תלות זאת חושבה לפני זה.

In [91]:
def P2O5_calc3_1(x):
  eff = x[0]
  entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
  bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entrain,0])
  # error1=abs((WS_P2O5-bat.yout[1]/98*71)/WS_P2O5)
  error=abs((Prod_P2O5-bat.xout[1]/98*71)/Prod_P2O5)
  print (eff, entrain, bat.yout[1]/98*71, bat.xout[1]/98*71, error, bat.runs, ',')

  return error

In [92]:
bounds=[(0.5,1)]
sol10_1=optimize.direct(P2O5_calc3_1, bounds=bounds, f_min=0, f_min_rtol=0.001, maxiter=100, locally_biased=False)
print (sol10_1.x, sol10_1.fun)

0.75 3.58232070311904 0.2316886366651532 37.53650385122883 0.0016887273609354266 40 ,
0.9166666666666666 3.7053610501815397 0.08588284141633287 37.88802595712421 0.0076602648171333 44 ,
0.5833333333333334 2.826956587995283 0.5529074478085162 36.53031845398312 0.028448977287682996 28 ,
0.8055555555555556 3.693592348591124 0.16624116668017097 37.72651921382196 0.003364872708030785 54 ,
0.6944444444444444 3.400790861195705 0.31045687166658387 37.20466528750617 0.010514221076963688 44 ,
0.9722222222222222 3.605858106299873 0.05825211702407979 37.830550460897186 0.006131661194074055 44 ,
0.861111111111111 3.7346057976119584 0.12033207394474967 37.8100317097481 0.005585949727343113 46 ,
0.7685185185185185 3.627217717882095 0.2084089109646816 37.616825219171986 0.0004474792332974582 42 ,
0.7314814814814815 3.529617222083623 0.25810388159773345 37.46173865645521 0.0036771633921486913 38 ,
[0.76851852] 0.0004474792332974582


In [93]:
eff10_1=sol10_1.x[0]
entrain10_1=entrain=(slope_proc-C[0]-C[2]*eff10_1-C[3]*eff10_1**2)/C[1]

optimal calcul

In [94]:
bat10_1=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff10_1, convergence=1E-3, entrainment_perc_out=[entrain10_1,0])
bat10_1.runs

42

In [95]:
bat=bat10_1
summary_table10_1['Model data']=[bat.Oin,bat.yin[1]/98*71,bat.Oout,bat.yout[1]/98*71,bat.Ain,bat.Aout,bat.xout[1]/98*71,f"{bat.Aout*bat.xout[1]/(bat.Oin*bat.yin[1]):.1%}", f"{bat.stages_num:.0f}", bat.eff[0,1], str(bat.EQUIL)[17:-2]]
summary_table10_1

,Plant data,Model data
PS_flow,159.196491,159.196491
PS_P2O5,11.6,11.6
WS_flow,134.196491,129.757894
WS_P2O5,0.2,0.208409
W_flow,23.4,23.4
Prod_flow,48.4,48.126303
Prod_P2O5,37.6,37.616825
P2O5_recovery,98.5%,98.0%
Stages_num,6,6
Efficiency,---,0.768519


In [96]:
table10_1=LLC.BatteryTable(bat10_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6
"Org in, ton/hr",159.196491,148.520452,141.340745,134.532371,132.074524,130.870701
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,10.915566,7.085996,3.090851,1.488594,0.704182
"y3 in, %",0.070000,0.088433,0.070263,0.038597,0.024596,0.019185
"Aq in, ton/hr",42.848001,35.404694,28.384919,25.758296,24.557021,23.400000
"x1 in, %",1.272653,1.272953,1.273526,1.494723,1.698311,0.000000
"x2 in, %",43.214779,33.078769,18.620093,9.006942,3.355499,0.000000
"x3 in, %",0.287008,0.268671,0.163577,0.082557,0.041054,0.020000
"Org out, ton/hr",148.530658,141.315647,134.531021,132.032818,130.908907,129.757894
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000


#### vizual

In [97]:
## P2O5
bat=bat10_1
summary_table=summary_table10_1
fig10_1 = go.Figure()
n=1 #H3PO4
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig10_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

In [98]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [99]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [100]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [101]:
fig_equil10_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat10_1
# H3PO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil10_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil10_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil10_1.update_scenes(camera_projection_type="orthographic")

fig_equil10_1.show()